In this notebook we use the vectorstore we created in the second notebook to ask questions and get response from a Llama2 model running locally using ollama. 

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [16]:
# # Create embeddingsclear
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

db = Chroma(persist_directory="./vecdb",
            embedding_function=embeddings)

# # Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

# # Create Ollama language model - llama2
local_llm = 'llama2'

In [17]:
llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=1024,  
                 temperature=0)

In [18]:
template = """<bos><start_of_turn>user\nAnswer the question based only on the following context and extract out a meaningful answer. \
Please write in full sentences with correct spelling and punctuation. if it makes sense use lists. \
If the context doen't contain the answer, just respond that you are unable to find an answer. \

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
# Create the RAG chain using LCEL with prompt printing and streaming output
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [20]:
# Function to ask questions
def ask_question(question):
    print("Answer:\n\n", end=" ", flush=True)
    for chunk in rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

In [21]:
while True:
    user_question = input("Ask a question (or type 'quit' to exit): ")
    if user_question.lower() == 'quit':
        break
    answer = ask_question(user_question)
    # print("\nFull answer received.\n")

Answer:

 The three gunas (qualities) mentioned in the context are:

1. Sattwa - the quality of purity, clarity, and peacefulness
2. Rajas - the quality of passion, activity, and energy
3. Tamas - the quality of darkness, ignorance, and inertia

These three gunas are considered to be the fundamental aspects of the universe and human nature, according to the Bhagavad Gita.

Answer:

 Based on the provided context, it seems that the passage is discussing the concept of materialistic pleasure and its effects on an individual's mind and spirit. The text suggests that pursuing materialistic pleasures can lead to a state of undone purpose, mind, and man, as well as a lack of knowledge of oneself. Additionally, the passage implies that not loving or hating objects of the sense can lead to a state of tranquility and healing of earthly pains.

In terms of the question you provided, it seems that the passage is warning against the dangers of pursuing materialistic pleasures, as it can lead to a 